In [46]:
using Cubature
using DelimitedFiles
using Interpolations
using Plots
using LinearAlgebra
using Optim

In [47]:
function covP(μ,k,b,f,G,n)
    cov = ((b + μ^2*f)^2*P(k)*G^2 + 1/n)^2
    return cov
end

covP (generic function with 1 method)

In [48]:
function dPda(μ,k,b,f,G,BAOonly=true)
    der = zeros(4)
    der[1] = 2*(b + μ^2*f)*P(k)
    der[2] = 2*μ^2*(b + μ^2*f)*P(k)
    # dP/da terms followed by d(b+mu^2*f)^2/da terms
    if BAOonly
        der[3] = (b + μ^2*f)^2*dPBAO(k)*μ^2*k
        der[4] = (b + μ^2*f)^2*dPBAO(k)*(1-μ^2)*k
    else
        der[3] = (b + μ^2*f)^2*dP(k)*μ^2*k + μ*(1 - μ^2)*2*(b + μ^2*f)*2*μ*f*P(k)
        der[4] = (b + μ^2*f)^2*dP(k)*(1-μ^2)*k - μ*(1 - μ^2)*2*(b + μ^2*f)*2*μ*f*P(k)
    end
    return der*G^2
end

dPda (generic function with 2 methods)

In [49]:
function Fisher(μ,k,b,f,G,Spar,Sper,n,i,j,BAOonly=true)
    result = dPda(μ,k,b,f,G,BAOonly)[i]*dPda(μ,k,b,f,G,BAOonly)[j]
    result /= covP(μ,k,b,f,G,n)
    result *= exp(-(1-μ^2)*k^2*Sper^2/2 -μ^2*k^2*Spar^2/2)
    return result
end

Fisher (generic function with 4 methods)

In [50]:
function forecast(b,f,G,Spar,Sper,n,V,BAOonly=true)
    kmin = 0.01
    kmax = 0.2
    μmin = 0
    μmax = 1
    fisher = zeros(4,4)
    for i=1:4
        for j=1:4        
            # dPda*icovP*dPda, x[1] is μ, x[2] is k        
            f_fisher(x) = Fisher(x[1],x[2],b,f,G,Spar,Sper,n,i,j,BAOonly)
            fisher[i,j] = hcubature(f_fisher,[μmin,kmin],[μmax,kmax], reltol=1e-3)[1]*V/(2*π)^2
        end
    end
    return fisher
end

forecast (generic function with 2 methods)

In [51]:
fish = forecast(2,0.8,0.6,10,8,1e-4,2.9e9,true)
print(100*sqrt.(diag(inv(fish[3:4,3:4]))))

[0.42271998415195433, 0.2888262317037433]

In [268]:
"""
Growth rate
"""
function growth(Om, z)
    f(x) = (Om*(1 + x)^3/(Om*(1 + x)^3 + 1 - Om))^0.55
    G = exp(-hquadrature(f, 0, z, reltol=1e-3)[1])
    return G, f(z)
end

growth

In [11]:
"""
Smooth Pk
"""
function smooth_P(k, par)
    keq, A, a0, a2, a4 = par
    q = k./keq
    L = log.(2*exp(1.0) .+ 1.8.*q)
    C = 14.2 .+ 731.0./(1 .+ 62.5.*q)
    T = L./(L .+ C.*q.^2)
    return A.*(k.*T.^2 .+ a0 + a2*k.^2 + a4*k.^4)
end

"""
Find best fit parameters of smooth Pk for Pt
"""
function best_smooth_P_params(k, Pt)
    chi2(x) = sum((Pt - smooth_P(k, x)).^2 .* k.^2)
    opt = Optim.minimizer(optimize(chi2, [0.6, 60000, 0, 0, 0]))
    return opt
end

best_smooth_P_params

In [269]:
"""
Effectiveness of reconstruction
"""
nP = [0, 0.2, 0.3, 0.5, 1.0, 2.0, 3.0, 6.0, 10.0]
r_factor = [1.0, 1.0, 0.9, 0.8, 0.70, 0.6, 0.55, 0.52, 0.5]
r_itp = interpolate((nP,), r_factor, Gridded(Linear()))
function recon(np)
    return r_itp(np)
end

recon (generic function with 1 method)

In [320]:
function damping(μ,k,z,np,Om,s8)
    G, f = growth(Om, z)
    S_per = 9.4*s8/0.9*G
    S_par = (1 + f)*S_per
    Dpar = μ^2*k^2*S_par^2
    Dper = (1-μ^2)*k^2*S_per^2
    rfact = recon(np)
    #rfact = 1
    Dfactor = exp(-rfact^2*(Dpar + Dper)/2)
    #Dfactor = 1
    return Dfactor
end 

damping (generic function with 2 methods)

In [12]:
Pk_class = readdlm("/mnt/c/Users/colkh/Documents/Programs/class_public/output/desi_pk.dat",comments=true)
P_itp = interpolate((Pk_class[:,1],), Pk_class[:,2], Gridded(Linear()))

function P(k)
    return P_itp(k)
end

k = collect(0.00005:0.00001:0.3)
par = best_smooth_P_params(k, P(k))
Pbao_itp = interpolate((k,), P(k) .- smooth_P(k, par), Gridded(Linear()))

function PBAO(k)
    return Pbao_itp(k)
end

function dP(k)
    return (P(k.+0.001) .- P(k))/0.001
end

function dPBAO(k)
    return (PBAO(k.+0.001) .- PBAO(k))/0.001
end

dPBAO (generic function with 1 method)

In [275]:
Vdesi = [2.63, 3.15, 3.65, 4.1, 4.52, 4.89, 5.22, 5.5, 5.75, 5.97, 6.15, 6.3, 6.43]*1e9
nPdesi = [6.23, 9.25, 5.98, 3.88, 1.95, 1.59, 1.41, 0.61, 0.53, 0.40, 0.22, 0.12, 0.12]
zdesi = [0.65,0.75,0.85,0.95,1.05,1.15,1.25,1.35,1.45,1.55,1.65,1.75,1.85]
Nelg = [309, 2269, 1923, 2094, 1441, 1353, 1337, 523, 466, 329, 126, 0.1, 0.1]
Nelg = Nelg*14000*0.1 ./Vdesi;

In [329]:
# my volume
s8 = 0.84
Om = 0.3
for i=1:length(Vdesi)
    G, f = growth(Om, zdesi[i])
    b = 0.84/G
    #V = volume_bin(zdesi[i]-0.05,zdesi[i]+0.05,Om)*14000/41252
    fish = forecast(b,f,G,P,dPBAO,Vdesi[i],10000,Om,zdesi[i],nPdesi[i],s8)
    #println(100*sqrt.(diag(inv(fish)[3:4,3:4])))
    e1, e2 = eigvals(inv(fish)[3:4,3:4])
    println(100*sqrt(e1*e2/(e1+e2)))
end

0.7534837687224551
0.669283220329987
0.6150367971526447
0.5751785924975725
0.5513601648605106
0.5270231540741982
0.5040694242021913
0.4991101011942829
0.48118704333626244
0.47023475066147796
0.4679881183268801
0.45599336234766813
0.4444487977293381


In [307]:
""" 
In Mpc/h 
"""
function distance(z, Om)
    return hquadrature(z -> 2998.0/sqrt(Om*(1 +z)^3 + 1 - Om), 0, z, reltol=1e-3)[1]
end

function volume_bin(zmin, zmax, Om)
    rmax = distance(zmax, Om)
    rmin = distance(zmin, Om)
    volume = 4*pi/3*(rmax^3 - rmin^3)
    return volume
end 

volume_bin (generic function with 1 method)

In [331]:
if not 2+2==5
    print("hey")
end

UndefVarError: UndefVarError: not not defined